# Tutoriel pytorch - TP3 - IFT725

Tel que mentionné dans l'énoncé du travail, vous devez recopier les blocs de code du tutoriel suivant

https://pytorch.org/tutorials/beginner/pytorch_with_examples.html

en donnant, pour chaque bloc, une description en format "markdown" de son contenu.

## Tensors
### Warm-up: numpy

In [1]:
import numpy as np

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# Randomly initialize weights
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    print(t, loss)

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # Update weights
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 27760881.993060984
1 22028944.841014035
2 20979138.80950557
3 21238226.159641333
4 20869871.184293743
5 18580527.46904524
6 14665217.217638683
7 10247629.805411965
8 6589887.737782886
9 4055845.2147009578
10 2518615.507584631
11 1630459.0407609558
12 1125048.7298997135
13 828757.2991339827
14 646567.3713471376
15 526675.8076935627
16 442063.07784285897
17 378659.41759274935
18 328805.9014803684
19 288256.2144152671
20 254405.81748203712
21 225698.83362930862
22 201062.95928098375
23 179725.898614112
24 161137.24788049422
25 144849.22970288782
26 130510.73434455905
27 117854.52496635231
28 106653.44022670685
29 96700.81113594574
30 87831.77199378537
31 79906.74591990445
32 72811.29705189774
33 66445.64588119621
34 60729.93385247389
35 55580.467083794356
36 50933.28951126883
37 46730.45915058401
38 42923.33169543728
39 39480.41476756854
40 36355.91648792876
41 33514.71022206578
42 30924.466988580196
43 28560.923408042778
44 26401.976164298223
45 24426.19889704937
46 22616.907509454846


370 0.0010979391840263645
371 0.001048589026285472
372 0.0010014810604862376
373 0.0009565000200183593
374 0.0009135468357440753
375 0.0008725468696272963
376 0.0008333878392810636
377 0.00079598804970645
378 0.000760280528791572
379 0.0007261931249551244
380 0.0006936313745532986
381 0.0006625436100179626
382 0.00063285050887517
383 0.0006044939413542275
384 0.0005774230767532369
385 0.0005515661911886984
386 0.0005268725301548421
387 0.0005032916989424362
388 0.0004807727358799478
389 0.00045926495249629996
390 0.00043872418903957857
391 0.00041910903591361396
392 0.0004003739339861378
393 0.0003824796167316725
394 0.0003653883451465383
395 0.0003490645050813658
396 0.0003334779050169585
397 0.00031858916193831685
398 0.00030436691113011654
399 0.0002907823177100352
400 0.0002778061523027081
401 0.00026541487092154555
402 0.00025357999970276895
403 0.0002422709856285937
404 0.00023147015223218132
405 0.00022115581411007312
406 0.00021129874930268789
407 0.00020188678695598396
408 0.0

#### Description des variables principales :
- x : batch de données en entrée de taille $\mathbb{R}^{64\times 1000}$
- y : cibles du batch de données de taille $\mathbb{R}^{64\times 10}$
- wi : matrices des poids de la couche i ($w_1 \in \mathbb{R}^{1000\times 100}$, $w_2 \in \mathbb{R}^{100\times 10}$)
- y_pred : la matrice des prédictions du batch de taille $\mathbb{R}^{64\times 10}$
- loss : valeur de la fonction de coût (i.e. la *loss*) pour le batch courant
- grad_wi : gradient de la loss par rapport à wi pour le batch courant (de même taille que wi)

#### Fonctionnement global du code :
- Réseau pleinement connecté à 2 couches prenant en entré des données dans $\mathbb{R}^{1000}$, avec une couche caché à 100
neuronnes et sortant des valeurs dans $\mathbb{R}^{10}$.
- Réalise 500 epochs d'un batch aléatoire de taille 64.
- Le réseau utilise ReLU pour la couche caché et arrays numpy pour la gestion des matrices.

### PyTorch: Tensors

In [2]:
import torch

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Randomly initialize weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 756.7928466796875
199 6.830591201782227
299 0.09496981650590897
399 0.0017424827674403787
499 0.000141580356284976


#### Description des variables principales :
- x : tenseur du batch de données en entrée
- y : tenseur des cibles du batch de données
- wi : tenseurs des poids de la couche i
- y_pred : le tenseur des prédictions du batch
- loss : valeur de la fonction de coût (i.e. la *loss*) pour le batch courant
- grad_wi : gradient de la loss par rapport à wi pour le batch courant

#### Fonctionnement global du code :
- Il s'agit du même réseau et des mêmes calculs que précédemment, mais cette fois ci avec l'utilisation des tenseurs
*PyTorch* plutôt que des arrays numpy.


## Autograd
### PyTorch: Tensors and autograd

In [3]:
import torch

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and outputs.
# Setting requires_grad=False indicates that we do not need to compute gradients
# with respect to these Tensors during the backward pass.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Create random Tensors for weights.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y using operations on Tensors; these
    # are exactly the same operations we used to compute the forward pass using
    # Tensors, but we do not need to keep references to intermediate values since
    # we are not implementing the backward pass by hand.
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call w1.grad and w2.grad will be Tensors holding the gradient
    # of the loss with respect to w1 and w2 respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    # An alternative way is to operate on weight.data and weight.grad.data.
    # Recall that tensor.data gives a tensor that shares the storage with
    # tensor, but doesn't track history.
    # You can also use torch.optim.SGD to achieve this.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()

99 171.9473114013672
199 0.2956036627292633
299 0.0011552288196980953
399 6.559419853147119e-05
499 1.8755792552838102e-05


#### Description des variables principales :
- x : tenseur du batch de données en entrée
- y : tenseur des cibles du batch de données
- wi : tenseurs des poids de la couche i
- y_pred : le tenseur des prédictions du batch
- loss : le tenseur de la fonction de coût (i.e. la *loss*) pour le batch courant

#### Fonctionnement global du code :
- Cette fois-ci, les tenseurs des poids w1 et w2 ont été modifié pour que le module `autograd` de PyTorch calcule
automatiquement le gradient par rapport à eux lors de la propagation arrière.
- Il n'y a plus besoin de stocker les résultats intermédiaire de la propagation avant.
- La loss est devenu un tenseur sur lequel il est possible d'effectuer la propagation arrière
- Les gradients sont récupéré via les attributs `grad` des tenseurs des poids w1 et w2.


### PyTorch: Defining new autograd functions

In [4]:
import torch


class MyReLU(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and outputs.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Create random Tensors for weights.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # To apply our Function, we use Function.apply method. We alias this as 'relu'.
    relu = MyReLU.apply

    # Forward pass: compute predicted y using operations; we compute
    # ReLU using our custom autograd operation.
    y_pred = relu(x.mm(w1)).mm(w2)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass.
    loss.backward()

    # Update weights using gradient descent
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()

99 1738.2916259765625
199 26.87868881225586
299 0.6236550807952881
399 0.01663106121122837
499 0.0007920173229649663


#### Description des variables principales :
- x : tenseur du batch de données en entrée
- y : tenseur des cibles du batch de données
- wi : tenseurs des poids de la couche i
- y_pred : le tenseur des prédictions du batch
- loss : le tenseur de la fonction de coût (i.e. la *loss*) pour le batch courant

#### Fonctionnement global du code :
- Cette fois, la fonction relu est une fonction personnalisé. Pour ce faire, il faut créer une classe dérivant de
`torch.autograd.Function` et implémenter les méthodes `forward` et `backward`
- Additionnelement, `ctx` est utilisé pour sauvegarder des variables lors de la propagation afin d'être réutilisé lors
de la propagation arrière.


## nn module
### PyTorch: nn

In [5]:
import torch

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. Each Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

99 2.1007936000823975
199 0.03723189979791641
299 0.0017959714168682694
399 0.00012242708180565387
499 9.246141416952014e-06


#### Description des variables principales :
- x : tenseur du batch de données en entrée
- y : tenseur des cibles du batch de données
- model : `torch.nn.Sequential` représentant le réseau pleinement connecté à deux couches et contenant le graphe
*computationel* ainsi que les gradients du réseau.
- y_pred : le tenseur des prédictions du batch
- loss_fn : `torch.nn.MSELoss` représentant l'erreur quadratique moyenne utilisé pour le réseau.
- loss : le tenseur de la fonction de coût (i.e. la loss) pour le batch courant

#### Fonctionnement global du code :
- Le module nn est désormais utilisé pour représenté le réseau à travers une séquence de couches.
- Il suffit ainsi d'appelé le model avec le batch pour calculer la prédiction.
- Il faut aussi remettre à zéro les gradients avant d'exécuter la propagation arrière.


### PyTorch: optim

In [6]:
import torch

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

99 62.09150695800781
199 1.223055124282837
299 0.010853434912860394
399 5.6448472605552524e-05
499 2.3283936911866476e-07


#### Description des variables principales :
- x : tenseur du batch de données en entrée.
- y : tenseur des cibles du batch de données.
- model : `torch.nn.Sequential` représentant le réseau pleinement connecté à deux couches.
- y_pred : le tenseur des prédictions du batch.
- loss_fn : `torch.nn.MSELoss` représentant l'erreur quadratique moyenne utilisé pour le réseau.
- loss : le tenseur de la fonction de coût (i.e. la loss) pour le batch courant.
- optimizer : classe représentant l'optimiseur Adam pour les paramètres du modèle.

#### Fonctionnement global du code :
- La remise à zéro des gradient est maintenant effectué sur l'optimiseur.
- Après l'exécution de la propagation arrière, la mise à jour des paramètres est effectué via la méthode `step` de
l'optimiseur. Il n'est plus nécessaire de ne pas grader l'historique dans `autograd` avec `torch.no_grad()`


### PyTorch: Custom nn Modules

In [7]:
import torch

class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred


# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Construct our model by instantiating the class defined above
model = TwoLayerNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 1.8625942468643188
199 0.019280346110463142
299 0.00042959119309671223
399 1.2642994079214986e-05
499 4.203012053949351e-07


#### Description des variables principales :
- x : tenseur du batch de données en entrée.
- y : tenseur des cibles du batch de données.
- model : `Module` personnalisé `TwoLayerNet` représentant le réseau pleinement connecté à deux couches.
- y_pred : le tenseur des prédictions du batch.
- criterion : `torch.nn.MSELoss` représentant l'erreur quadratique moyenne utilisé pour le réseau.
- loss : le tenseur de la fonction de coût (i.e. la loss) pour le batch courant.
- optimizer : classe représentant l'optimiseur SGD pour les paramètres du modèle.

#### Fonctionnement global du code :
- Les seuls changements concernent le model qui est désormais une classe personnalisé dérivant `torch.nn.Module` et 
l'optimiseur qui est redevenu SGD.
- Le réseau et toutes les autres opérations sont les mêmes.


### PyTorch: Control Flow + Weight Sharing

In [8]:
import random
import torch


class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we construct three nn.Linear instances that we will use
        in the forward pass.
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 0, 1, 2, or 3
        and reuse the middle_linear Module that many times to compute hidden layer
        representations.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same Module many
        times when defining a computational graph. This is a big improvement from Lua
        Torch, where each Module could be used only once.
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred


# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Construct our model by instantiating the class defined above
model = DynamicNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 14.59460163116455
199 6.151984214782715
299 1.522809624671936
399 0.4552117586135864
499 0.25161540508270264


#### Description des variables principales :
- x : tenseur du batch de données en entrée.
- y : tenseur des cibles du batch de données.
- model : `Module` personnalisé `DynamicNet` représentant un réseau pleinement connecté avec un nombre aléatoire de
couches cachés.
- y_pred : le tenseur des prédictions du batch.
- criterion : `torch.nn.MSELoss` représentant l'erreur quadratique moyenne utilisé pour le réseau.
- loss : le tenseur de la fonction de coût (i.e. la loss) pour le batch courant.
- optimizer : classe représentant l'optimiseur SGD pour les paramètres du modèle.

#### Fonctionnement global du code :
- Le modèle est encore une fois ici changé pour un `Module` personnalisé dynamique. Cette implémentation démontre
certaines fonctionnalités que Lua Torch ne possède pas : en particulier, la capacité de réutiliser un même module
plusieurs fois.
- Du momentum a aussi été rajouté sur la SGD.